#Run these cells to install dependencies
These may need to be edited as YOLOv5 and these dependencies are constantly being updated

In [ ]:
from tensorflow import keras
import tensorflow_datasets as tfds
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import tensorflow as tf
import numpy as np
import glob
from shutil import copyfile
from PIL import Image
from math import floor, ceil
import sys

In [ ]:
#This is a customized version of YOLOv5, allowing edits to the C3 modules within the YOLOv5 head
!git clone https://github.com/achelm15/yolov5.git
%cd yolov5
!pip install -r requirements.txt

import os
import cv2
import zipfile
from os import path
import shutil
import csv
import random
import numpy as np

# The next few cells can be used to download and train on LPCV Data Directly
# You must upload all LPCV M4V and CSV files to create the dataset

In [ ]:
%mkdir LPCV
%cd LPCV
%mkdir images
%mkdir labels
%mkdir valid
%mkdir valid/images
%mkdir valid/labels
%mkdir train
%mkdir train/images
%mkdir train/labels
%cd ..

/content/yolov5/LPCV
/content/yolov5


In [ ]:
%rm -r LPCV/train
%rm -r LPCV/valid
%mkdir ./LPCV/valid
%mkdir ./LPCV/valid/images
%mkdir ./LPCV/valid/labels
%mkdir ./LPCV/train
%mkdir ./LPCV/train/images
%mkdir ./LPCV/train/labels

In [ ]:
def capture_frames(video, variety):
  vidcap = cv2.VideoCapture(video)
  success,image = vidcap.read()
  count = 0
  while success:
    cv2.imwrite("LPCV/images/{}_frame{}.jpg".format(variety,count), image)     # save frame as JPEG file      
    success,image = vidcap.read()
    count += 1
  print("Finished Reading " + video)

In [ ]:
def create_labels(video_csv, variety):
  with open(video_csv, newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    yolo_labels = []
    for row in csv_reader:
      frame = row[0]
      if row[0] == "Frame":
        continue
      else:
        with open("LPCV/labels/"+variety+"_frame" + str(frame) + ".txt", "a") as yololabelfile:
          yoloarray = row
          del yoloarray[2:3]
          del yoloarray[0:1]
          for item in yoloarray:
            yololabelfile.write("%s " % item)
          yololabelfile.write("\n")

In [ ]:
capture_frames('4p1b_01A2.m4v','4p1b')
capture_frames('5p2b_01A1.m4v','5p2b')
capture_frames('5p4b_01A2.m4v','5p4b')
capture_frames('5p5b_03A1.m4v','5p5b')
capture_frames('7p3b_02M.m4v','7p3b')

In [ ]:
create_labels('4p1b_01A2.csv','4p1b')
create_labels('5p2b_01A1.csv','5p2b')
create_labels('5p4b_01A2.csv','5p4b')
create_labels('5p5b_03A1.csv','5p5b')
create_labels('7p3b_02M.csv','7p3b')

In [ ]:
for root, dirs, files in os.walk('LPCV/labels/', topdown=False):
    for filename in files:
      num = filename[10:filename.find(".")]
      name = filename[:10]
      video_file = name+num+".jpg"
      copyfile('LPCV/labels/'+filename, 'LPCV/train/labels/'+filename)
      copyfile('LPCV/images/'+video_file, 'LPCV/train/images/'+video_file)

In [ ]:
!zip -r LPCVtrain.zip

#To save LPCV image and labels, zip and download the LPCV folder at this point. In the future the folder can be uploaded and the above cells are uneccessary

In [ ]:
%%capture
%rm -r ./LPCV

In [ ]:
%%capture
!unzip LPCVtrain.zip

In [ ]:
%mkdir ./LPCV/valid
%mkdir ./LPCV/valid/images
%mkdir ./LPCV/valid/labels

In [ ]:
def contrast_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.random_contrast(img, 2, 5, 42)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    copy_label(image, "contrast", folder)
def noise_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.stateless_random_jpeg_quality(img, 2, 20, (42,42))
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    copy_label(image, "noise", folder)
def grayscale_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.rgb_to_grayscale(img)
    img = img.numpy()
    img = img.reshape((img.shape[0], img.shape[1]))
    im = Image.fromarray(img, 'L')
    im.save(folder+'/images/'+new_image)
    copy_label(image, "grayscale", folder)
def saturate_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.random_saturation(img, 2, 120, 42)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    copy_label(image, "saturate", folder)
def hue_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.random_hue(img, 0.1)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    copy_label(image, "hue", folder)
def flip_image_lr(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.flip_left_right(img)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    flip_label_lr(image[:image.find('.')]+".txt", image[:image.find('.')]+"_flip_lr.txt", folder)
def flip_image_ud(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.flip_up_down(img)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    flip_label_ud(image[:image.find('.')]+".txt", image[:image.find('.')]+"_flip_ud.txt", folder)
def rot_90(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.rot90(img)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    label_rot_90(image[:image.find('.')]+".txt", image[:image.find('.')]+"_rot_90.txt", folder)
def crop_bbox(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    width = img.shape[1]
    height = img.shape[0]
    label_file = folder+"/labels/"+image[:image.find('.')]+".txt"
    ball_list = []
    with open(label_file) as f:
        count = 0
        lines = f.readlines()
        new_lines = []
        for x in lines:
            words = x.split(" ")
            if words[0]=="1":
                ball_list.append(words)
                xp = float(words[1])
                yp = float(words[2])
                wp = float(words[3])
                hp = float(words[4])
                x1 = int(floor(xp*width-(wp*width)/2))
                y1 = int(floor(yp*height-(hp*height)/2))
                image = tf.image.crop_to_bounding_box(img,y1,x1,int(ceil(hp*height)),int(ceil(wp*width)))
                image = image.numpy()
                im = Image.fromarray(image)
                file = new_image[0:new_image.find('.')]+"_"+str(count)+new_image[new_image.find('.'):]
                im.save(folder+'/images/'+file)
                new_label_file = file[:file.find('.')]+".txt"
                bb_crop_label(new_label_file, folder)
                count += 1
def bb_crop_label(new_label_file, folder):
    with open(folder+'/labels/'+new_label_file, 'w') as f:
        f.write("1 0.5 0.5 1.0 1.0")
def flip_label_lr(label_file, new_label_file, folder):
    label_file = folder+"/labels/"+label_file
    new_label_file = folder+"/labels/"+new_label_file
    with open(label_file) as f:
        lines = f.readlines()
        new_lines = []
        for x in lines:
            words = x.split(" ")
            words[1] = str(1-float(words[1]))
            new_lines.append(" ".join(words))
    f.close()
    with open(new_label_file, 'w') as f:
        for x in new_lines:
            f.write(x)
def flip_label_ud(label_file, new_label_file, folder):
    label_file = folder+"/labels/"+label_file
    new_label_file = folder+"/labels/"+new_label_file
    with open(label_file) as f:
        lines = f.readlines()
        new_lines = []
        for x in lines:
            words = x.split(" ")
            words[2] = str(1-float(words[2]))
            new_lines.append(" ".join(words))
    f.close()
    with open(new_label_file, 'w') as f:
        for x in new_lines:
            f.write(x)
def label_rot_90(label_file, new_label_file, folder):
    label_file = folder+"/labels/"+label_file
    new_label_file = folder+"/labels/"+new_label_file
    with open(label_file) as f:
        lines = f.readlines()
        new_lines = []
        for x in lines:
            words = x.split(" ")
            x = words[1]
            y = words[2]
            w = words[3]
            h = words[4]
            words[1] = y
            words[2] = str(1-float(x))
            words[3] = h
            words[4] = w
            new_lines.append(" ".join(words))
    f.close()
    with open(new_label_file, 'w') as f:
        for x in new_lines:
            f.write(x)
def copy_label(image, augmentation, folder):
    filename = image[:image.find('.')]
    new_filename = folder+'/labels/'+filename+'_'+augmentation+'.txt'
    filename = folder+'/labels/'+filename+'.txt'
    copyfile(filename, new_filename)

In [ ]:
aug_list = ['_contrast', '_noise', '_flip_lr', '_flip_ud', '_rot_90', '_saturate', '_hue']
data_dir = "./LPCV/train"

In [ ]:
for root, dirs, files in os.walk(data_dir+'/images/', topdown=False):
    for file in files:
      count = 0
      for j in aug_list:
        filename = file[:file.find('.')]+j+file[file.find('.'):]
        if count==0: contrast_image(file, filename, data_dir)
        if count==1: noise_image(file, filename, data_dir)
        if count==2: flip_image_lr(file, filename, data_dir)
        if count==3: flip_image_ud(file, filename, data_dir)
        if count==4: rot_90(file, filename, data_dir)
        if count==5: saturate_image(file, filename, data_dir)
        if count==6: hue_image(file, filename, data_dir)
        count+=1

In [ ]:
for root, dirs, files in os.walk(data_dir+'/images/', topdown=False):
    for file in files:
        filename = file[:file.find('.')]+"_grayscale"+file[file.find('.'):]
        grayscale_image(file, filename, data_dir)

In [ ]:
def moveFiles(src, dst):
  files = []
  for i in os.listdir(src):
    files.append(i)
  files = sorted(files)
  listt = np.random.RandomState(seed=12).permutation(files)[:300]
  for f in listt:
    shutil.copy(os.path.join(src, f), dst)
    p = os.path.join(src, f)
    os.remove(p)

In [ ]:
data = open("data.yaml", "w")
data.write("train: ./LPCV/train/images\nval: ./LPCV/valid/images\nnc: 2\nnames: [\'person\', \'sports-ball\']")
data.close()

In [ ]:
%cp -r LPCV LPCVrestart

#Copy directory in case of need for later use

#Add additional Ball pictures

Additional images downloaded from: https://purdue0-my.sharepoint.com/personal/akocher_purdue_edu/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fakocher%5Fpurdue%5Fedu%2FDocuments%2FVIP%2FDataset%2FballPerson%2Etar%2Egz&parent=%2Fpersonal%2Fakocher%5Fpurdue%5Fedu%2FDocuments%2FVIP%2FDataset&ga=1

The file at this link should be downloaded and placed in your google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cp ../gdrive/MyDrive/Fall2021/ballPerson.tar.gz ballPerson.tar.gz  

In [ ]:
%rm -r ballPerson

In [ ]:
!tar -xf ballPerson.tar.gz

In [ ]:
def moveFiles(src, dst):
  files = []
  for i in os.listdir(src):
    files.append(i)
  files = sorted(files)
  listt = np.random.RandomState(seed=12).permutation(files)[:750]
  for f in listt:
    shutil.copy(os.path.join(src, f), dst)
    p = os.path.join(src, f)
    os.remove(p)

In [ ]:
moveFiles("ballPerson/images/train", "LPCV/train/images")
moveFiles("ballPerson/labels/train", "LPCV/train/labels")

In [ ]:

def moveFiles2(src, dst):
  files = []
  for i in os.listdir(src):
    files.append(i)
  files = sorted(files)
  listt = np.random.RandomState(seed=12).permutation(files)[:250]
  for f in listt:
    shutil.copy(os.path.join(src, f), dst)
    p = os.path.join(src, f)
    os.remove(p)

In [ ]:
moveFiles2("ballPerson/images/train", "LPCV/valid/images")
moveFiles2("ballPerson/labels/train", "LPCV/valid/labels")

In [ ]:
%ls LPCV/train/images | wc -l
%ls LPCV/train/labels | wc -l
%ls LPCV/valid/images | wc -l
%ls LPCV/valid/labels | wc -l

In [ ]:
def moveFiles(src, dst):
  files = []
  for i in os.listdir(src):
    files.append(i)
  files = sorted(files)
  listt = np.random.RandomState(seed=12).permutation(files)[:300]
  for f in listt:
    shutil.copy(os.path.join(src, f), dst)
    p = os.path.join(src, f)
    os.remove(p)
moveFiles("LPCV3/train/images", "LPCV3/valid/images")
moveFiles("LPCV3/train/labels", "LPCV3/valid/labels")

#The next cell can be used to train the model, adjust parameters as desired

In [ ]:
!python train.py --batch 50 --img 320 --data data.yaml --project modified --freeze 1 3 5 7 9 --changes 2,4,6,8,13 --cfg ./models/yolov5s.yaml --weights yolov5s.pt --hyp data/hyps/hyp.finetune.yaml 

#The next cell can be used to test the model

In [ ]:
!python val.py --weights modified/train/exp/weights/best.tflite --img 320 --data data.yaml